In [69]:
import os
import pandas as pd
import joblib

In [70]:
data_path = os.path.join('../data/interim')
raw_data_path = os.path.join('../data/raw/ml-100k')
model_path = os.path.join('../models')

In [71]:
model = joblib.load(os.path.join(model_path, "best_xgb_model.pkl"))

In [72]:
u_data = pd.read_csv(os.path.join(raw_data_path, "u.data"), sep="\t", header=None)
u_data.columns = ["user_id", "item_id", "rating", "timestamp"]
item = pd.read_csv(os.path.join(raw_data_path, "u.item"), sep="|", encoding='latin-1', header=None)
item = item[[0,1]]
item.columns = ["item_id", "title"]
user = pd.read_csv(os.path.join(raw_data_path, "u.user"), sep= "|", encoding='latin-1', header=None, index_col=0, names=["id", "age", "gender", "occupation", "zip_code"])

In [73]:
processed_data= pd.read_csv(os.path.join(data_path, "preprocessed_data.csv"), sep="|")

In [134]:
def recommend_movies(user_ids, data, model, titles, n =5):
    data = data[data["user_id"].isin(user_ids)]
    pred_ratings = model.predict(data)
    pred_ratings = pd.Series(pred_ratings,  name="rating")
    data = pd.concat([data.reset_index(), pred_ratings], axis=1)
    data = pd.merge(data, titles, on=["item_id"], how='left')
    data = data[["user_id", "title", "rating"]]
    data = data.sort_values(by="rating", ascending=False)
    data = data.groupby(by=["user_id"]).head(n)
    return data



In [135]:
recommend_movies(u_data["user_id"][:5], processed_data, model, item[["item_id", "title"]])

,user_id,title,rating
433,244,"Shawshank Redemption, The (1994)",4.406176
452,244,"Wrong Trousers, The (1993)",4.365980
446,244,Star Wars (1977),4.350738
435,244,Schindler's List (1993),4.321645
434,244,Raiders of the Lost Ark (1981),4.312913
227,186,"Usual Suspects, The (1995)",4.265660
208,186,"Silence of the Lambs, The (1991)",4.250646
209,186,Pulp Fiction (1994),4.094324
267,196,Secrets & Lies (1996),4.092527
183,186,Fargo (1996),4.072012
